# Лабораторная работа по теме Итерационные методы решения слау

### Вариант 4
Метод наискорейшего градиентного спуска

##### описание метода
Метод наискорейшего градиентного спуска применяется для решения системы линейных алгебраических уравнений (СЛАУ) вида: $Ax=b$ \
где \
A - симметричная положительно определенная матрица n×n \
x — вектор неизвестных \
b — вектор правых частей

In [110]:
import numpy as np

In [111]:
def solve(
    A: np.ndarray,
    b: np.ndarray,
    eps: float = 1e-15,
) -> tuple[np.ndarray]:
    """
    Решение системы методом наискорейшего градиентного спуска
    residual - невязка
    """
    
    """
    этап 1:
    готовим начальные x и невязку
    для отслеживания насколько близко мы находимся к решению
    """
    n = A.shape[0]
    x = np.zeros(n)
    x_new = np.ones(n)
    residual = A @ x - b
    residual_new = residual

    it_counter = 0

    while np.all(abs(x - x_new) > eps):
        it_counter += 1
        x = x_new
        residual = residual_new
        """
        этап 2:
        вычисляем AResidual - насколько матрица A "усиливает" вектор невязки для расчёта оптимального шага alpha
        Вычисление оптимальный шаг alpha
        и обновляем решение
        """
        AResidual = A @ residual
        alpha = np.dot(residual, residual) / np.dot(residual, AResidual)
        x_new = x - alpha * residual
        
        """
        этап 3:
        вычисляем новую невязку по новому решению
        """
        residual_new = A @ x_new - b

        if it_counter % 100 == 0:
            print(f"{it_counter=}: residual={np.linalg.norm(residual_new):.6e}")

    print(f"Итоги: {it_counter=}, residual={np.linalg.norm(residual_new):.6e}")
    
    return x
    

In [112]:
def check_input_matrix(A: np.ndarray, b: np.ndarray) -> bool:
    eigvals = np.linalg.eigvals(A)
    if np.any(eigvals <= 0):
        return False
    
    return np.all(np.linalg.solve(A, b))

In [113]:
def generate(size: int = 4, tryings: int = 10**7) -> tuple[np.ndarray, np.ndarray]:
    for _ in range(tryings):
        # A_random = np.random.randn(size, size)
        A_random = np.random.randint(-10, 10, size=(size, size))
        A = (A_random + A_random.T)

        b = np.random.randint(-10, 10, size=(size))
        if check_input_matrix(A, b):
            return A, b

In [114]:
def test(A: np.ndarray, b: np.ndarray) -> None:
    print(f"исходная матрица A {A}")
    print(f"исходный вектор b {b}")
    print(f"проверка на валидность входных данных {check_input_matrix(A, b)}")

    my_x = solve(A, b)
    np_x = np.linalg.solve(A, b)
    error = abs(my_x - np_x)

    print(
        f"мой x: {my_x}",
        f"точный x: {np_x}",
        f"Ошибка: {error}",
        f"Максимальная ошибка: {np.max(error):.6e}",
        sep="\n"
    )

In [115]:
# Тест: Простая симметричная матрица
A = np.array([
    [2, 1],
    [1, 2],
], dtype=float)
b = np.array([1,1])
test(A, b)

исходная матрица A [[2. 1.]
 [1. 2.]]
исходный вектор b [1 1]
проверка на валидность входных данных True
Итоги: it_counter=3, residual=0.000000e+00
мой x: [0.33333333 0.33333333]
точный x: [0.33333333 0.33333333]
Ошибка: [1.11022302e-16 5.55111512e-17]
Максимальная ошибка: 1.110223e-16


In [116]:
A, b = generate(size=5)
test(A, b)

исходная матрица A [[  4   1   1   5  -3]
 [  1   4   5   0   0]
 [  1   5  10   4  -3]
 [  5   0   4  16 -10]
 [ -3   0  -3 -10  18]]
исходный вектор b [ 9 -6 -4 -2 -8]
проверка на валидность входных данных True
it_counter=100: residual=5.607659e-01
it_counter=200: residual=3.456028e-02
it_counter=300: residual=2.129968e-03
it_counter=400: residual=1.312710e-04
it_counter=500: residual=8.090295e-06
it_counter=600: residual=4.986089e-07
it_counter=700: residual=3.072952e-08
it_counter=800: residual=1.893884e-09
it_counter=900: residual=1.167075e-10
it_counter=1000: residual=7.198166e-12
it_counter=1100: residual=4.618024e-13
Итоги: it_counter=1124, residual=2.275488e-13
мой x: [  9.65217391 -12.17391304   6.60869565  -5.17391304  -0.60869565]
точный x: [  9.65217391 -12.17391304   6.60869565  -5.17391304  -0.60869565]
Ошибка: [1.93622895e-13 3.32178729e-13 1.98951966e-13 1.14575016e-13
 9.54791801e-15]
Максимальная ошибка: 3.321787e-13
